## LangChain Memory
Storing and summarizing conversation history in structurized form.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
# message history
history = ChatMessageHistory()

history.add_user_message("Buenos dias!")
history.add_ai_message("hello!")
history.add_user_message("Whats your name?")
history.add_ai_message("My name is GIGACHAT")
history.messages

[HumanMessage(content='Buenos dias!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My name is GIGACHAT', additional_kwargs={}, response_metadata={})]

In [3]:
# memory
# History keeps all messages between the user and AI intact. History is what the user sees in the UI.
# It represents what was actually said. Memory keeps some information, which is presented to the LLM to make it behave as if it "remembers" the conversation.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Buenos dias!")
memory.chat_memory.add_ai_message("Hello!")
memory.chat_memory.add_user_message("Whats your name?")
memory.chat_memory.add_ai_message("My name is GIGACHAT")
memory.load_memory_variables({})

/tmp/ipykernel_20864/1306437153.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: Buenos dias!\nAI: Hello!\nHuman: Whats your name?\nAI: My name is GIGACHAT'}

In [4]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)
conversation.predict(input="Buenos Dias!") # will it response in different language?



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Buenos Dias!
AI:


/tmp/ipykernel_20864/3004657571.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
/tmp/ipykernel_20864/3004657571.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(



> Finished chain.


" Hello! It's currently 9:00 AM here in my location. I am an AI programmed to respond to human interactions. How can I assist you today?"

## Summary

In [5]:
!pip install tiktoken

In [6]:
from langchain.memory import ConversationSummaryBufferMemory

previous_output_review = "Rdr2 is an experience, as it's more than just a video game. It's like being in one long Oscar winning movie. Script, acting, storyline are immense, it's quite unbelievable really. You are living as a cowboy day in day out eating, bathing, sleeping, everything in a huge openworld."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1-0)
memory.save_context(
    {"input": f"Could you analyze review for me {previous_output_review}?"},
    {"output": "Sure, no problem"},
)

/tmp/ipykernel_20864/3499675901.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1-0)


In [7]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [8]:
conversation.predict(input="Thank you")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asks the AI to analyze a review for the game Rdr2, which is described as an immersive experience similar to living in an Oscar-winning movie. The AI agrees to do so.
Human: Thank you
AI:

> Finished chain.


" You're welcome! I am always happy to assist with any tasks related to analyzing reviews. I have a vast database of information on various games, including Rdr2. Did you know that Rdr2 stands for Red Dead Redemption 2, and it was released in 2018 by Rockstar Games? It received critical acclaim for its immersive open-world gameplay and compelling story. It even won several Game of the Year awards, including one from the Academy of Interactive Arts & Sciences."